In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io
import itertools
import json
import pathlib
import requests
import en_core_web_md
import os
import re
import missingno as msno
import spacy
import textacy
from html.parser import HTMLParser
import sys
sys.path.insert(0, '/Users/nissani/Desktop/msvdd_Bloc/msvdd_bloc')
import msvdd_bloc

In [63]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [64]:
%watermark -v -iv

numpy          1.17.2
json           2.0.9
textacy        0.9.1
re             2.2.1
pandas         0.25.1
matplotlib     3.1.1
seaborn        0.9.0
missingno      0.4.2
spacy          2.1.8
requests       2.22.0
en_core_web_md 2.1.0
CPython 3.7.0
IPython 7.8.0


In [65]:
data_dir = pathlib.Path("/Users/nissani/Desktop/msvdd_Bloc/data/postings")
role_queries = [
    "web developer",
    "front-end developer",
    "back-end developer",
    "full-stack developer",
    "python",
    "data scientist",
    "data engineer",
]
category_queries = ["Engineering", "Data Science"]
locations = [
    "New York, NY",
    "Chicago, IL",
    "San Francisco, CA",
    "San Jose, CA",
    "Los Angeles, CA",
    "Seattle, WA",
    "Austin, TX",
    "Washington, DC",
    "Baltimore, MD",
    "Boston, MA",
]

In [66]:
def load_dataset(filepath, **kwargs):
    """
    Args:
        filepath (str)
        kwargs: Additional params passed as-is
            into :func:`pd.read_json()`, useful for setting
            dataset-specific dtypes, etc.
    
    Returns:
        :class:`pd.DataFrame`
    """
    # almost too easy ;)
    return pd.read_json(path_or_buf=filepath, **kwargs)


def inspect_dataset(dataset):
    """
    Args:
        dataset (:class:`pd.DataFrame`)
    """
    print("shape:", dataset.shape)
    print("columns:", ", ".join(dataset.columns))
    return dataset.describe(include="all")


class _HTMLParser(HTMLParser):
    
    def __init__(self):
        super().__init__()
        self._texts = []
    
    def handle_data(self, data):
        self._texts.append(data)
        
    def get_text(self):
        return "".join(self._texts)


def extract_text_from_html(html):
    """
    Basic, stdlib-only function for extracting text data from HML.
    
    Args:
        html (str)
        
    Returns:
        str
    """
    parser = _HTMLParser()
    parser.feed(html)
    parser.close()
    return parser.get_text().strip()

In [79]:
github_jobs = load_dataset('/Users/nissani/Desktop/msvdd_Bloc/data/postings/github_jobs.json')
indeed_jobs = load_dataset('/Users/nissani/Desktop/msvdd_Bloc/data/postings/indeed_jobs.json')
muse_jobs = load_dataset('/Users/nissani/Desktop/msvdd_Bloc/data/postings/themuse_jobs.json')

In [80]:
github_jobs['description'][0]

'<p>Simon &amp; Schuster is seeking a Lead Software Engineer to join a rapidly growing team focused on impacting the world of publishing through research and innovation. Working with a team of data scientists, data engineers, designers, and domain experts, you will be involved in rapidly prototyping, developing, and deploying the platforms that put insights and information into the hands of decision-makers.</p>\n<p>From databases to serverless applications, you will be designing, deploying , and managing the systems that connect our real-world data to the books and authors that inform and entertain our world. As a Full Stack (Engineer|Developer) you will engage with engineers and domain experts across the organization, collaborating to design and deploy innovative new products and services. From inception to production, you will onboard, integrate and deploy an array of cutting-edge tools and technologies to seamlessly deliver our solutions.</p>\n<p>Publishing experience is not require

Remove from github: url, id, company_url, company_log

Keep from github: type, created_at, company, location, title, description, how_to_apply

In [81]:
print(github_jobs.columns)
github_standard = github_jobs.drop(['url', 'id', 'company_url', 'company_logo', 'how_to_apply', 'type'], axis=1)
github_standard.head()

Index(['id', 'type', 'url', 'created_at', 'company', 'company_url', 'location',
       'title', 'description', 'how_to_apply', 'company_logo'],
      dtype='object')


,created_at,company,location,title,description
0,2019-09-11 18:59:59+00:00,Simon & Schuster,"New York, NY",Lead Software Engineer,<p>Simon &amp; Schuster is seeking a Lead Soft...
1,2019-09-09 15:29:25+00:00,Noe & Associates,"New York, NY",Web Developer,<p>A leading international branding and design...
2,2018-11-19 19:40:01+00:00,Prosek Partners,"Fairfield, CT",Front End Developer,"<p>Background</p>\n<p>Prosek Partners, one of ..."
3,2018-11-19 15:44:44+00:00,Aon Cyber Solutions,NY or LA,Cybersecurity Firm Seeks Senior Developer,<p>Aon Cyber Solutions is looking for a Senior...
4,2018-11-14 22:16:02+00:00,The Atlantic,"Washington, DC & New York, NY",Lead Front End Developer,<p>The Atlantic is a digital-first media compa...


Remove from Indeed: city, state, country, source, url, onmousedown, latitude, longitude, jobkey, sponsored, indeedApply, formattedLocation, stations, recommendations

Keep from Indeed: jobtitle, company, snippet, formattedLocationFull, formattedRelativeTime, date

In [87]:
print(indeed_jobs.columns)
indeed_standard = indeed_jobs.drop(['city', 'state', 'country', 'source', 'url', 'onmousedown', 'latitude', 'longitude', 
                                'jobkey', 'sponsored', 'indeedApply', 'formattedLocation', 'stations', 'recommendations',
                                'language', 'expired', 'formattedRelativeTime'], axis = 1)
indeed_standard.head()

Index(['jobtitle', 'company', 'city', 'state', 'country', 'language',
       'formattedLocation', 'source', 'date', 'snippet', 'url', 'onmousedown',
       'latitude', 'longitude', 'jobkey', 'sponsored', 'expired',
       'indeedApply', 'formattedLocationFull', 'formattedRelativeTime',
       'stations', 'recommendations'],
      dtype='object')


,jobtitle,company,date,snippet,formattedLocationFull
0,Junior Developer,Muratek,2019-09-20 16:25:56+00:00,"Develop and maintain code, image assets and co...","New York, NY 10038"
1,Front End Developer,Halo Media,2019-09-11 00:50:51+00:00,Our staff works on accounts by some of the wor...,"New York, NY 10001"
2,Mid-Level Fullstack Javascript Engineer,Mantl,2019-09-20 22:53:35+00:00,Our mission is to use technology to level the ...,"New York, NY"
3,Entry Level Software Developer,CloudServiceTek,2019-09-27 23:37:05+00:00,Sound expertise in defining technical architec...,"New York, NY 10020"
4,Junior Software Developer Intern,Live Building Systems,2019-09-25 15:08:35+00:00,Completed or working on Bachelor’s Degree in C...,"New York, NY 10010"


Remove from Muse: type, short_name, model_type, id, tags, refs

Keep from Muse: contents, name, publication_date, locations, categories, levels, company(name)

In [83]:
print(muse_jobs.columns)
muse_standard = muse_jobs.drop(['type', 'short_name', 'model_type', 'id', 'tags', 'refs', 'levels', 'categories'], axis = 1)
muse_standard.head()

Index(['contents', 'name', 'type', 'publication_date', 'short_name',
       'model_type', 'id', 'locations', 'categories', 'levels', 'tags', 'refs',
       'company'],
      dtype='object')


,contents,name,publication_date,locations,company
0,<p><span>Language diverse monorepo seeks exper...,Lead Build Engineer,2019-08-19T14:30:34.532418Z,"[{'name': 'New York, NY'}]","{'id': 11755, 'short_name': 'beeswax', 'name':..."
1,<p><span>In a Site Reliability Engineering rol...,Site Reliability Engineer,2019-08-19T14:30:34.597968Z,"[{'name': 'New York, NY'}]","{'id': 11755, 'short_name': 'beeswax', 'name':..."
2,<div><span>Etsy is looking for an experienced ...,Core Platform Engineering Manager,2019-07-23T18:38:29.951266Z,"[{'name': 'New York, NY'}]","{'id': 1222, 'short_name': 'etsy', 'name': 'Et..."
3,<h3>The company:</h3>\n\n<p>We're on a mission...,Engineering Team Lead - Cloud Platforms,2019-08-30T17:06:09.667397Z,"[{'name': 'New York, NY'}]","{'id': 756, 'short_name': 'datadog', 'name': '..."
4,<p><strong>About This Position </strong></p>\n...,Senior Software Engineer,2019-06-19T07:56:17.584313Z,"[{'name': 'New York, NY'}]","{'id': 1507, 'short_name': 'ellevest', 'name':..."


In [88]:
muse_standard.columns = ['description', 'job_title', 'date_posted', 'location', 'company']
github_standard.columns = ['date_posted', 'company', 'location', 'job_title', 'description']
indeed_standard.columns = ['job_title', 'company', 'date_posted', 'description', 'location']

In [91]:
def reformat(dataframe):
    list_of_titles = ['job_title', 'company', 'location', 'description', 'date_posted']
    data = []
    for el in list_of_titles:
        data.append(dataframe[el])
        
    new_dataframe = pd.concat(data, axis = 1)
    
    new_dataframe.columns = list_of_titles
    
    return new_dataframe

In [94]:
muse = reformat(muse_standard)
github = reformat(github_standard)
indeed = reformat(indeed_standard)

In [95]:
all_jobs = pd.concat([github, muse, indeed])

In [101]:
all_jobs.index = [i for i in range(len(all_jobs))]

In [104]:
all_jobs.head()

,job_title,company,location,description,date_posted
0,Lead Software Engineer,Simon & Schuster,"New York, NY",<p>Simon &amp; Schuster is seeking a Lead Soft...,2019-09-11 18:59:59+00:00
1,Web Developer,Noe & Associates,"New York, NY",<p>A leading international branding and design...,2019-09-09 15:29:25+00:00
2,Front End Developer,Prosek Partners,"Fairfield, CT","<p>Background</p>\n<p>Prosek Partners, one of ...",2018-11-19 19:40:01+00:00
3,Cybersecurity Firm Seeks Senior Developer,Aon Cyber Solutions,NY or LA,<p>Aon Cyber Solutions is looking for a Senior...,2018-11-19 15:44:44+00:00
4,Lead Front End Developer,The Atlantic,"Washington, DC & New York, NY",<p>The Atlantic is a digital-first media compa...,2018-11-14 22:16:02+00:00
